In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.autograd import Variable

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import cv2

from PIL import Image
import copy
from collections import namedtuple
import os
import random
import shutil
import time

In [0]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

In [0]:
filepath = '/content/covid-chestxray-dataset/metadata.csv'
imagepath = '/content/covid-chestxray-dataset/images/'

In [0]:
df = pd.read_csv(filepath)
print(df.shape)

In [0]:
df.head(3)

In [0]:
l = df['finding'].unique()

In [0]:
l

In [0]:

positive_images = []#positive images
filename = []
for(i,row) in df.iterrows():
  if row["finding"] == "COVID-19" and row["view"]=="PA":
    filename.append(row["filename"])
    # print(filename)

In [0]:
for f in filename:
  positive_images.append(imagepath+f)

In [0]:
import os
import random 

In [0]:
path_normal = '/content/drive/My Drive/normal'
normal_images = os.listdir(path_normal)
random.shuffle(normal_images)
norml_images  = []#normal images
for i in range(len(positive_images)):
  norml_images.append(path_normal+'/'+normal_images[i])

In [0]:
norml_images[76]

We are considering equal number of positive and negative cases

In [0]:
label = []#saving labels in a list
leng = 2*len(positive_images)
for i in range(leng):
  label.append(1)

for i in range(int(leng/2)):
  label[i+int(leng/2)] = 0

In [0]:
imagepaths = []#appending paths of all images
for imag in positive_images:
  imagepaths.append(imag)

for imag in norml_images:
  imagepaths.append(imag)

In [0]:
classes = {1:'Corona',0:'Normal'}

In [0]:
import torchvision.transforms as transforms
pretrained_size = 224
train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.RandomResizedCrop(224),
                           transforms.ToTensor()
                       ])

In [0]:
class newdataset(Dataset):
  
  def __init__(self,imagepaths,labels,transform=None):
    self.imagelist = imagepaths
    self.label = labels
    self.transform = transform

  def __len__(self):
    return len(self.imagelist)

  def __getitem__(self,index):
    image_path = self.imagelist[index]
    img = Image.open(image_path).convert('RGB')
    img = self.transform(img)
    lab = self.label[index]

    return img,lab

In [0]:
dataset = newdataset(imagepaths,label,transform=train_transforms)

In [0]:
ratio = 0.8
n_train_examples = int(len(dataset) * ratio)
n_valid_examples = len(dataset) - n_train_examples

train,test = torch.utils.data.random_split(dataset, [n_train_examples, n_valid_examples])

In [0]:
val_ratio = 0.5
val_examples = int(len(test)*val_ratio)
test_examples = len(test)-val_examples

val,final_test = torch.utils.data.random_split(test,[val_examples,test_examples])

In [0]:
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(train,shuffle = True, batch_size = BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val,shuffle = True, batch_size = BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test,shuffle = True, batch_size = BATCH_SIZE)

In [0]:
class Xraynet(nn.Module):
  def __init__(self):
    super(Xraynet,self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1,bias=True)
    self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, bias=True)
    self.max_pool_1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv3 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, bias=True)
    self.max_pool_2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, bias=True)
    self.max_pool_3 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv5 = torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, bias=True)
    self.max_pool_4 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1 = nn.Linear(in_features=18432, out_features=4000)   #Flattened image is fed into linear NN and reduced to half size
    self.droput = nn.Dropout(p=0.5)                    #Dropout used to reduce overfitting
    self.fc2 = nn.Linear(in_features=4000, out_features=2000)
    self.droput = nn.Dropout(p=0.5)
    self.fc3 = nn.Linear(in_features=2000, out_features=500)
    self.droput = nn.Dropout(p=0.5)
    self.fc4 = nn.Linear(in_features=500, out_features=50)
    self.droput = nn.Dropout(p=0.5)
    self.fc5 = nn.Linear(in_features=50, out_features=2)     
    self.relu = nn.ReLU() 

    self.dropout_1 = torch.nn.Dropout(0.25)


  def forward(self,x):
    x = torch.nn.functional.relu(self.conv1(x))
    x = torch.nn.functional.relu(self.conv2(x))
    x = self.max_pool_1(x)
    x = self.dropout_1(x)

    x = torch.nn.functional.relu(self.conv3(x))
    x = self.max_pool_2(x)
    x = self.dropout_1(x)

    x = torch.nn.functional.relu(self.conv4(x))
    x = self.max_pool_3(x)
    x = self.dropout_1(x)

    x = torch.nn.functional.relu(self.conv5(x))
    x = self.max_pool_4(x)
    x = self.dropout_1(x)

    x = x.view(-1,18432)
    out = self.fc1(x)
    out = self.relu(out)
    out = self.droput(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.droput(out)
    out = self.fc3(out)
    out = self.relu(out)
    out = self.droput(out)
    out = self.fc4(out)
    out = self.relu(out)
    out = self.droput(out)
    out = self.fc5(out)
    return out

In [0]:
torch.cuda.empty_cache()

In [0]:
model = Xraynet()
# defining the optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=2e-4,betas=(0.9, 0.99),weight_decay=0.0005)
# defining the loss function
criterion = torch.nn.CrossEntropyLoss()
# checking if GPU is available
torch.cuda.empty_cache()
if torch.cuda.is_available():
    model = model.cuda()

print(model)

In [0]:
torch.cuda.empty_cache()

In [0]:
def trainmodel(epoch):
  
  model.train()
  for i in range(epoch):
    
    tr_loss = 0
    best_loss = 0
    best_loss = 9999
    optimizer.zero_grad()
    for x,target in train_loader:
      x, target = Variable(x), Variable(target)
      
      output_train = model(x.float().cuda())
      _, preds = torch.max(output_train.data, 1)
      target = target.to('cpu')
      preds = preds.to('cpu')
      
      print(preds==target.data)
      
      loss_train = criterion(output_train,target.cuda())
      train_losses.append(loss_train)
      loss_train.backward()
      optimizer.step()
      tr_loss = loss_train.item()
      
    with torch.no_grad():
      running_loss = 0
      for val,val_target in val_loader:
        
        val,val_target = Variable(val), Variable(val_target)
        output_val = model(val.float().cuda())
        
        loss_val = criterion(output_val, val_target.cuda())
        running_loss += loss_val.item()
        
        if(loss_val.item()<best_loss):
          best_loss = loss_val
          bestweights = model.state_dict()

        val_losses.append(loss_val)
        
      print('Epoch : ',i+1, '\t', 'loss :', running_loss/len(val))

  model.load_state_dict(bestweights) 
  return model   


In [0]:
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
Model = trainmodel(n_epochs)

In [0]:
model.eval()
running_corrects = 0
for img,out in test_loader:
  img,out = Variable(img), Variable(out)
  output_val = Model(img.float().cuda())
  _, preds = torch.max(output_val.data, 1)
  
  out = out.to('cpu')
  preds = preds.to('cpu')
  running_corrects += torch.sum(preds == out.data).item()

test_Accuracy = running_corrects/len(test)
print(test_Accuracy)